# RAID LAB Hiring Project

In [1]:
#IMPORTANT
#this chunk lets you have multiple outputs from a single chunk; run it first!
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [2]:
import numpy as np
import pandas as pd

import gurobipy as gp
from gurobipy import GRB

# tested with Python 3.7.0 & Gurobi 9.0

In [3]:
# Create dictionaries to capture factory supply limits, depot throughput limits, and customer demand.

supply = dict({'P1': 500000})

through = dict({'D1': 70000,
                'D2': 50000,
                'D3': 100000,
                'D4': 40000,
                'D5': 250000})

demand = dict({'C1': 50000,
               'C2': 10000,
               'C3': 40000,
               'C4': 35000,
               'C5': 60000,
               'C6': 20000,
               'C7': 45000,
               'C8': 52000,
               'C9': 65000,
               'C10': 27000})

dots, penalty = gp.multidict({
               ('C1'): 100,
               ('C2'): 100,
               ('C3'): 100,
               ('C4'): 100,
               ('C5'): 100,
               ('C6'): 100,
               ('C7'): 100,
               ('C8'): 100,
               ('C9'): 100,
               ('C10'): 100})

# Create a dictionary to capture shipping costs.

arcs, cost = gp.multidict({
    ('P1', 'D1'): 0.5,
    ('P1', 'D2'): 0.5,
    ('P1', 'D3'): 1.0,
    ('P1', 'D4'): 0.2,
    ('P1', 'D5'): 0.2,
    ('D1', 'C1'): 2.0,
    ('D2', 'C2'): 1.5,
    ('D2', 'C3'): 0.5,
    ('D2', 'C4'): 1.5,
    ('D2', 'C6'): 1.0,
    ('D2', 'C7'): 1.0,
    ('D2', 'C8'): 1.0,
    ('D2', 'C9'): 1.0,
    ('D2', 'C10'): 1.0,    
    ('D3', 'C1'): 1.0,
    ('D3', 'C2'): 0.5,
    ('D3', 'C3'): 0.5,
    ('D3', 'C4'): 1.0,
    ('D3', 'C5'): 0.5,
    ('D4', 'C2'): 1.5,
    ('D4', 'C3'): 2.0,
    ('D4', 'C5'): 0.5,
    ('D4', 'C6'): 1.5,
    ('D4', 'C7'): 1.5,
    ('D4', 'C8'): 1.5,
    ('D4', 'C9'): 1.5,
    ('D4', 'C10'): 1.5,    
    ('D5', 'C3'): 0.2,
    ('D5', 'C4'): 1.5,
    ('D5', 'C5'): 0.5,
    ('D5', 'C6'): 1.5,
    ('D5', 'C7'): 1.5,
    ('D5', 'C8'): 1.5,
    ('D5', 'C9'): 1.5,
    ('D5', 'C10'): 1.5
})

In [4]:
model = gp.Model('SupplyNetworkDesign')
flow = model.addVars(arcs, obj=cost, name="flow")
deficit = model.addVars(dots, obj = penalty, name = 'deficit')
model.update()

Restricted license - for non-production use only - expires 2022-01-13


In [5]:
# Production capacity limits

producers = supply.keys()
producer_flow = model.addConstrs((gp.quicksum(flow.select(producer, '*')) <= supply[producer]
                                 for producer in producers), name="producer")

In [6]:
# Customer demand

customers = demand.keys()
customer_flow = model.addConstrs((gp.quicksum(flow.select('*', customer)+deficit.select(customer)) == demand[customer]
                                  for customer in customers), name="customer")

In [7]:
# Depot flow conservation

depots = through.keys()
depot_flow = model.addConstrs((gp.quicksum(flow.select(depot, '*')) == gp.quicksum(flow.select('*', depot))
                               for depot in depots), name="depot")

In [8]:
# The second set limits the product passing through the depot to be at most equal the throughput of that deport.

# Depot throughput

depot_capacity = model.addConstrs((gp.quicksum(flow.select('*', depot)) <= through[depot]
                                   for depot in depots), name="depot_capacity")

In [9]:
model.optimize()

Gurobi Optimizer version 9.1.2 build v9.1.2rc0 (win64)
Thread count: 6 physical cores, 12 logical processors, using up to 12 threads
Optimize a model with 21 rows, 45 columns and 85 nonzeros
Model fingerprint: 0x3c444e4c
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [2e-01, 1e+02]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+04, 5e+05]
Presolve removed 7 rows and 2 columns
Presolve time: 0.01s
Presolved: 14 rows, 43 columns, 72 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    3.3700000e+05   5.261250e+04   0.000000e+00      0s
      18    5.7900000e+05   0.000000e+00   0.000000e+00      0s

Solved in 18 iterations and 0.01 seconds
Optimal objective  5.790000000e+05


In [10]:
obj = model.getObjective()
print('Objective Equation Function:')
obj

print('\n Objective Function Value:')
print(obj.getValue())

print('\n Vaccine Flow Values: \n')

product_flow = pd.DataFrame(columns=["From", "To", "Flow", "Cost"])
for arc in arcs:
    if flow[arc].x > 1e-6:
        product_flow = product_flow.append({"From": arc[0], "To": arc[1], "Flow": flow[arc].x, "Cost": flow[arc].x*cost[arc]}, ignore_index=True)  

product_flow.index=[''] * len(product_flow)
product_flow


print('\n Defecit Values: \n')

deficit_flow = pd.DataFrame(columns=["Customer", "Deficit", "Penalty"])
for customer in customers:
    if deficit[customer].x > -1e-6:
        deficit_flow = deficit_flow.append({"Customer": customer, "Deficit": deficit[customer].x, "Penalty": deficit[customer].x*penalty[customer]}, ignore_index=True)

deficit_flow.index=[''] * len(deficit_flow)
deficit_flow

Objective Equation Function:


<gurobi.LinExpr: 0.5 flow[P1,D1] + 0.5 flow[P1,D2] + flow[P1,D3] + 0.2 flow[P1,D4] + 0.2 flow[P1,D5] + 2.0 flow[D1,C1] + 1.5 flow[D2,C2] + 0.5 flow[D2,C3] + 1.5 flow[D2,C4] + flow[D2,C6] + flow[D2,C7] + flow[D2,C8] + flow[D2,C9] + flow[D2,C10] + flow[D3,C1] + 0.5 flow[D3,C2] + 0.5 flow[D3,C3] + flow[D3,C4] + 0.5 flow[D3,C5] + 1.5 flow[D4,C2] + 2.0 flow[D4,C3] + 0.5 flow[D4,C5] + 1.5 flow[D4,C6] + 1.5 flow[D4,C7] + 1.5 flow[D4,C8] + 1.5 flow[D4,C9] + 1.5 flow[D4,C10] + 0.2 flow[D5,C3] + 1.5 flow[D5,C4] + 0.5 flow[D5,C5] + 1.5 flow[D5,C6] + 1.5 flow[D5,C7] + 1.5 flow[D5,C8] + 1.5 flow[D5,C9] + 1.5 flow[D5,C10] + 100.0 deficit[C1] + 100.0 deficit[C2] + 100.0 deficit[C3] + 100.0 deficit[C4] + 100.0 deficit[C5] + 100.0 deficit[C6] + 100.0 deficit[C7] + 100.0 deficit[C8] + 100.0 deficit[C9] + 100.0 deficit[C10]>


 Objective Function Value:
579000.0

 Vaccine Flow Values: 



,From,To,Flow,Cost
,P1,D2,50000.0,25000.0
,P1,D3,64000.0,64000.0
,P1,D4,40000.0,8000.0
,P1,D5,250000.0,50000.0
,D2,C9,50000.0,50000.0
,D3,C1,50000.0,50000.0
,D3,C2,10000.0,5000.0
,D3,C4,4000.0,4000.0
,D4,C7,25000.0,37500.0
,D4,C9,15000.0,22500.0



 Defecit Values: 



,Customer,Deficit,Penalty
,C1,0.0,0.0
,C2,0.0,0.0
,C3,0.0,0.0
,C4,0.0,0.0
,C5,0.0,0.0
,C6,0.0,0.0
,C7,0.0,0.0
,C8,0.0,0.0
,C9,0.0,0.0
,C10,0.0,0.0


In [11]:
df_supply = pd.read_excel ('covid_input_example.xlsx', sheet_name = 'supply')
df_through = pd.read_excel ('covid_input_example.xlsx', sheet_name = 'through')
df_demand = pd.read_excel ('covid_input_example.xlsx', sheet_name = 'demand')
df_penalty = pd.read_excel ('covid_input_example.xlsx', sheet_name = 'penalty')
df_cost = pd.read_excel ('covid_input_example.xlsx', sheet_name = 'cost')

# Testing conversion from Excel
df_demand

# Testing transformation of parameters into dictionary from Stack Overflow https://stackoverflow.com/questions/26716616/convert-a-pandas-dataframe-to-a-dictionary
#demand #the original dictionary
demand1 = df_demand.set_index('Customer').T.to_dict('records')[0]
demand1

# Completing the rest

supply1 = df_supply.set_index('Producer').T.to_dict('records')[0]
#supply1
through1 = df_through.set_index('Depot').T.to_dict('records')[0]
#through1

FileNotFoundError: [Errno 2] No such file or directory: 'covid_input_example.xlsx'

In [25]:
# converting the Arcs to a Gurobi Multidictionary

#First Converting to Numpy Array
penalty1a = df_penalty.to_numpy()
penalty1a


# Iterating over the numpy array to convert to Gurobi Multidictionary
dots1, penalty1 = gp.multidict({row[0]: row[1] for row in penalty1a})
penalty1


#Repeating for Flow Costs
cost1a = df_cost.to_numpy()
#cost1a

arcs1, cost1 = gp.multidict({(row[0],row[1]): row[2] for row in cost1a})
#cost1

NameError: name 'df_penalty' is not defined

In [ ]:
#Creating the Same Exact Model and adding the 1 suffix to the model, variables, and constraints


model1 = gp.Model('SupplyNetworkDesign1')
flow1 = model1.addVars(arcs1, obj=cost1, name="flow1")
deficit1 = model1.addVars(dots1, obj = penalty1, name = 'deficit1')
model1.update()

# Production capacity limits

producers1 = supply1.keys()
producer_flow1 = model1.addConstrs((gp.quicksum(flow1.select(producer, '*')) <= supply1[producer]
                                 for producer in producers1), name="producer1")


# Customer demand

customers1 = demand1.keys()

customer_flow1 = model1.addConstrs((gp.quicksum(flow1.select('*', customer)+deficit1.select(customer)) == demand1[customer]
                                  for customer in customers1), name="customer1")

# Depot flow conservation

depots1 = through1.keys()
depot_flow1 = model1.addConstrs((gp.quicksum(flow1.select(depot, '*')) == gp.quicksum(flow1.select('*', depot))
                               for depot in depots), name="depot1")

# Depot throughput

depot_capacity1 = model1.addConstrs((gp.quicksum(flow1.select('*', depot)) <= through1[depot]
                                   for depot in depots1), name="depot_capacity1")

model1.optimize()

In [ ]:
obj1 = model1.getObjective()
print('Objective Equation Function:')
obj1

print('\n Note this is the exact same objective function as before, the variables are the same as well \n')

print('\n Objective Function Value:')
print(obj1.getValue())

print('\n Vaccine Flow Values: \n')

product_flow1 = pd.DataFrame(columns=["From", "To", "Flow", "Cost"])
for arc in arcs1:
    if flow1[arc].x > 1e-6:
        product_flow1 = product_flow1.append({"From": arc[0], "To": arc[1], "Flow": flow1[arc].x, "Cost": flow1[arc].x*cost1[arc]}, ignore_index=True)  

product_flow1.index=[''] * len(product_flow1)
product_flow1


print('\n Defecit Values: \n')

deficit_flow1 = pd.DataFrame(columns=["Customer", "Deficit", "Penalty"])
for customer in customers1:
    if deficit1[customer].x > -1e-6:
        deficit_flow1 = deficit_flow1.append({"Customer": customer, "Deficit": deficit1[customer].x, "Penalty": deficit1[customer].x*penalty1[customer]}, ignore_index=True)

deficit_flow1.index=[''] * len(deficit_flow1)
deficit_flow1